In [7]:
import requests
from bs4 import BeautifulSoup
import time

url = 'https://dev.to/'
response = requests.get(url)

if response.status_code == 200:
    soup = BeautifulSoup(response.text, 'lxml')

    # Find the navigation links
    nav_links = soup.find('ul', class_='default-navigation-links')

    # Check if navigation links are found
    if nav_links:
        # Find all list items within the navigation links
        list_items = nav_links.find_all('li')

        # Search for the specific link with text "/tags"
        tag_url = [item.a['href'] for item in list_items if item.a and item.a['href'] == '/tags']

        # Combine the URL with the tags relative path
        if tag_url:
            full_tag_url = url + tag_url[0]
            response = requests.get(full_tag_url)
            soup = BeautifulSoup(response.text, 'lxml')

            # Find div links
            div_links = soup.find_all('div', class_='js-tag-card')
            if div_links:
                anchor_tags = [div.find('a') for div in div_links if div.find('a')]

                # Extract and print the href attributes of the anchor tags
                anchor_hrefs = [tag['href'] for tag in anchor_tags if tag]

                # Merge the URL with the hrefs extracted from their anchor tags
                full_category_links = [url + href for href in anchor_hrefs]

            for category in full_category_links:
                response = requests.get(category)
                soup = BeautifulSoup(response.text, 'lxml')

                # Phishing for links that get to the articles
                article_div = soup.find_all('div', class_='crayons-story__indention')
                if article_div:
                    divs = [div.find('a') for div in article_div if div.find('a')]

                    # Extract the hrefs in the anchor tags to achieve our purpose: extract articles
                    article_href = [url + tag['href'] for tag in divs if tag]

                    for article_link in article_href:
                        response = requests.get(article_link)
                        soup = BeautifulSoup(response.text, 'lxml')

                        # Scraping the articles now!!!
                        article_body = soup.find_all('div', class_='crayons-article__main')

                        # Assuming there's only one div with class 'crayons-article__main'
                        if article_body:
                            article_content = article_body[0].get_text()

                            # Write the article content to a text file
                            with open('articles.txt', 'a', encoding='utf-8') as file:
                                file.write(article_content + '\n' + '-' * 150 + '\n')  # Add a line of dashes after each article

                            # Introduce a delay of 1 second between requests
                            time.sleep(1)
